In [1]:
import re
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import datetime
import arrow
from ics import Calendar, Event

In [2]:
# 学籍番号
student_id = "s" + "*******"

# パスワード
password = "********"

In [3]:
# chrome driverの設定
options = ChromeOptions()
options.add_argument("--headless")
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(
    options=options,
    service=ChromeService(ChromeDriverManager().install())
)

# urlを指定
url="https://muscat.musashino-u.ac.jp/portal/top.do"
driver.get(url)

# 学籍番号を入力
driver.find_element(By.XPATH,"//*[@id='userId']").send_keys(student_id)

# パスワードを入力
driver.find_element(By.XPATH,"//*[@id='password']").send_keys(password)

# ログインボタンをクリック
btn = driver.find_element(By.XPATH,"//*[@id='loginButton']")
btn.click()

# メニューの非表示を解除
menuList = driver.find_element(
    By.XPATH,
    "/html/body/div[1]/div[1]/div[6]/ul/li[3]/ul"
)
driver.execute_script(
    "arguments[0].setAttribute('style','display: block;')",
    menuList
)

# 履修登録確認にアクセス
myCalendar = driver.find_element(
    By.XPATH,
    "/html/body/div[1]/div[1]/div[6]/ul/li[3]/ul/li[1]/a"
)
myCalendar.send_keys(Keys.ENTER)

time.sleep(1)

# 時間割を抽出
jikanwari = []
xpaths = [
    "/html/body/div/div[2]/table/tbody/tr/td[1]/form/div[1]/div[4]/div[2]/div[3]/",
    "/html/body/div/div[2]/table/tbody/tr/td[1]/form/div[1]/div[5]/div[3]/",
    "/html/body/div/div[2]/table/tbody/tr/td[1]/form/div[2]/div[3]/",
    "/html/body/div/div[2]/table/tbody/tr/td[1]/form/div[4]/div[3]/",
]
for xpath in xpaths:
    table = []
    for i in range(3, 16, 2):
        for j in range(3, 14, 2):
            coma_xpath = xpath + f"table/tbody/tr[{i}]/td[{j}]/table/tbody/tr/td"
            coma = driver.find_element(By.XPATH, coma_xpath)
            coma_text = coma.get_attribute("innerHTML")
            pattern = r'<br>(.*?)<br>'
            matches = re.findall(pattern, coma_text, re.DOTALL)
            table.append(matches[0].strip())
    jikanwari_ = [
        table[0:6], table[6:12], table[12:18],
        table[18:24], table[24:30], table[30:36], table[36:42]
    ]
    jikanwari.append(jikanwari_)

# chromeを終了
driver.close()

In [4]:
jikanwari

[[['', '', '', '', '', ''],
  ['', '', '', '', '', ''],
  ['', '', '', '', '卒業論文Ⅰ', ''],
  ['', '', '', '', '卒業論文Ⅰ', ''],
  ['', '', '', '', '', ''],
  ['', '', '', '', '', ''],
  ['', '', '', '', '', '']],
 [['', '時空間データベース', '', '', '', ''],
  ['', '時空間データベース', '', '', '', ''],
  ['ロボティクス・IoT', '', '', '', '卒業論文Ⅰ', ''],
  ['ロボティクス・IoT', '', '', '', '卒業論文Ⅰ', ''],
  ['', '', '', '', '', ''],
  ['', '', '', '', '', ''],
  ['', '', '', '', '', '']],
 [['', '', '', '', '', ''],
  ['', '', '', '', '', ''],
  ['', '', '', '', '卒業論文Ⅱ', ''],
  ['', '', '', '', '卒業論文Ⅱ', ''],
  ['', '', '', '', '', ''],
  ['', '', '', '', '', ''],
  ['', '', '', '', '', '']],
 [['', '', '', '', '', ''],
  ['', '', '人工知能(AI)社会の情報倫理', '', '', ''],
  ['マーケティング・オートメーション', '', '人工知能(AI)社会の情報倫理', '', '卒業論文Ⅱ', ''],
  ['マーケティング・オートメーション', '', '', '', '卒業論文Ⅱ', ''],
  ['', '', '', '', '', ''],
  ['', '', '', '', '', ''],
  ['', '', '', '', '', '']]]

In [7]:
# カレンダーの作成
cal = Calendar()
cal.creator = 'Rintaro Fukui'

# 学期
semester = {
    0: [datetime.date(2024,4,15), datetime.date(2024,6,8)],
    1: [datetime.date(2024,6,13), datetime.date(2024,7,31)],
    2: [datetime.date(2024,9,20), datetime.date(2024,11,11)],
    3: [datetime.date(2024,11,19), datetime.date(2025,1,25)],
}

# 休日
holiday = [
    datetime.date(2024,5,1), datetime.date(2024,5,2),
    datetime.date(2024,5,3), datetime.date(2024,5,4),
    datetime.date(2024,5,6), datetime.date(2024,5,21),
    datetime.date(2024,10,11), datetime.date(2024,10,12),
    datetime.date(2024,10,14), datetime.date(2024,11,15),
    datetime.date(2024,11,16), datetime.date(2024,11,18),
    datetime.date(2024,12,11), datetime.date(2024,12,24),
    datetime.date(2024,12,25), datetime.date(2024,12,26),
    datetime.date(2024,12,27), datetime.date(2024,12,28),
    datetime.date(2024,12,30), datetime.date(2024,12,31),
    datetime.date(2025,1,1), datetime.date(2025,1,2),
    datetime.date(2025,1,3), datetime.date(2025,1,4),
    datetime.date(2025,1,13), datetime.date(2025,1,14),
    datetime.date(2025,1,17), datetime.date(2025,1,18),
]

# 時間
class_time = {
    0: ['08:50:00', '10:30:00'],
    1: ['10:40:00', '12:20:00'],
    2: ['13:10:00', '14:50:00'],
    3: ['15:00:00', '16:40:00'],
    4: ['16:50:00', '18:30:00'],
    5: ['18:40:00', '20:20:00'],
    6: ['20:20:00', '22:00:00'],
}

# 時間割に合わせてイベントを作成
for i in range(4):

    # 学期の開始日
    start = semester[i][0]

    # 学期の終了日
    stop = semester[i][1]

    # 学期の時間割
    jikanwari_ = jikanwari[i]

    while start <= stop:

        # 曜日を取得
        weekday = start.weekday()

        for i in range(len(jikanwari_)):

            rele = jikanwari_[i]

            # 日曜日ではない/授業がある/休日ではない を全て満たす場合にイベントを作成
            if weekday != 6 and rele[weekday] != '' and start not in holiday:

                # イベントを作成
                event = Event()

                # イベント名
                event.name = jikanwari_[i][weekday]

                # 開始時刻
                event.begin = arrow.get(
                    f"{start} {class_time[i][0]}",
                    "YYYY-MM-DD HH:mm:ss"
                ).replace(tzinfo="Asia/Tokyo")

                # 終了時刻
                event.end = arrow.get(
                    f"{start} {class_time[i][1]}",
                    "YYYY-MM-DD HH:mm:ss"
                ).replace(tzinfo="Asia/Tokyo")

                # イベントをカレンダーに追加
                cal.events.add(event)

        start += datetime.timedelta(days=1)

In [8]:
# 保存
with open('my.ics', 'w', encoding='utf-8') as f:
    f.write(str(cal))